In [24]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://finred.usalearning.gov/Money/CreditHistory")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [25]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [26]:
question = "What is credit ? "
docs = vectorstore.similarity_search(question)
len(docs)

4

In [27]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
)

In [ ]:
response_message = model.invoke(
    "talk about credit"
)

print(response_message.content)

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'Here are the main themes extracted from the retrieved documents:\n\n1. **Task Decomposition**: Breaking down complex tasks into smaller, manageable subgoals to facilitate efficient handling.\n2. **Planning**: Identifying and planning ahead for multiple steps involved in a task, including:\n\t* Subgoal identification\n\t* Task execution\n\t* Reflection and refinement (learning from mistakes and improving future results)\n3. **LLM-Powered Autonomous Agent System**: Using Large Language Models (LLMs) to enable autonomous systems to perform tasks with human inputs, simple prompting, or task-specific instructions.\n\nThese themes highlight the importance of breaking down complex tasks into manageable parts, planning ahead, and using LLMs as a key component in achieving autonomy in task execution.'

In [15]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "what is current price tesla?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

"I don't know. The provided context does not contain information about Tesla's stock price or any financial data. It appears to be related to artificial intelligence and autonomous agents, specifically a neuro-symbolic architecture for autonomous agents called MRKL."

In [16]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [18]:
question = "what is current price tesla?"

qa_chain.invoke(question)

"I don't know. The retrieved context does not contain information about Tesla's current price."